# DDPLKO Moduł 12 - praca domowa - Wdrażanie modelu

To dwunasta praca domowa w Programie szkoleniowym Klasyfikacja obrazu od Deep Drive PL

Twoim zadaniem w tym module będzie:
- [ ] Wdrożenie jednego ze swoich modeli jako aplikację demo: modelu do klasyfikacji rysunków z QuickDraw, modelu wytrenowanego z wykorzystaniem transfer learningu, modelu do klasyfikacji binarnej
- [ ] Wykorzystaj np. FastAPI, Streamlit lub Gradio App
- [ ] Udostępnij screenshot wdrożonej aplikacji na Discordzie `#klasyfikacja-wyniki`

Extra - dodatkowo możesz:
- Skorzystać z technik takich jak Torch Serve, TensorFlow Serving czy Nvidia Triton Inference server

Jako rozwiązanie prześlij kod źródłowy z którego korzystasz - notebook/skrypt

In [1]:
import gradio as gr

import tensorflow as tf
import numpy as np
from PIL import Image

import os
import tensorflow_addons as tfa

In [2]:
classes = os.listdir('/media/jakub/D/BarkDataset')

In [3]:
def build_model(img_size,num_classes):
    inputs = tf.keras.layers.Input(shape=(img_size, img_size, 3))
    
    base_model = tf.keras.applications.ResNet50(include_top=False, 
                                                        input_tensor=inputs, weights="imagenet")

    # New top
    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(base_model.output)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2, name="top_dropout")(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")(x)

    model = tf.keras.Model(inputs, outputs, name="ResNet50")
    optimizer = tfa.optimizers.AdamW(learning_rate=1e-5, weight_decay=1e-4)
    model.compile(
        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"],
        run_eagerly=True
    )
    return model

model = build_model(224, len(classes))
model.load_weights('/home/jakub/BarkClassifier/weights.06-0.33_resnet50_2.hdf5')

In [6]:
def predict(image):
    image = Image.fromarray(image.astype('uint8'), 'RGB')
    image = image.resize((224, 224))
    batch = np.expand_dims(image, 0)
    batch = tf.keras.applications.resnet.preprocess_input(batch) / 255.0
    pred = model.predict(batch)[0]
    return {classes[i]: float(pred[i]) for i in range(len(classes))}

In [8]:
imagein = gr.inputs.Image()
label = gr.outputs.Label(num_top_classes=5)
cls = gr.outputs.Textbox(type="auto", label=None)

images = [["BOJ_1.jpg"],
          ["BOP_1.jpg"],
          ["CHR_1.jpg"],
          ['EPB_1.jpg'],
          ['EPN_1.jpg']]

gr.Interface([predict], 
            imagein, 
            label,
            title="ResNet50 Tree Bark Classification",
            description="Bark Classification based on https://github.com/ulaval-damas/tree-bark-classification",
            examples=images).launch();

Running on local URL:  http://127.0.0.1:7862/

To create a public link, set `share=True` in `launch()`.


# Wyślij rozwiązanie
Możesz skorzystać z jednego z poniższych sposobów:
**mailem na specjalny adres** ze strony pracy domowej w panelu programu prześlij jedno z poniższych:
- notebooka (jeżeli plik ma mniej niż np. 10MB)
- notebooka w zipie
- link do Colaba (udostępniony)
- link do pliku przez GDrive/Dropboxa/WeTransfer/...
- pdfa (poprzez download as pdf)
- jako plik w repozytorium na np. GitHubie, by budować swoje portfolio (wtedy uważaj na wielkość pliku, najlepiej kilka MB, Max 25MB)

Najlepiej, by w notebooku było widać wyniki uruchomienia komórek, chyba, że przez nie plik będzie mieć 100+MB wtedy najlepiej Colab lub jakieś przemyślenie co poszło nie tak (zbyt dużo dużych zdjęć wyświetlonych w komórkach).

## Co otrzymasz?
Informację zwrotną z ewentualnymi sugestiami, komentarzami.